In [ ]:
import json
import pandas as pd
import os

In [ ]:
lang_pair = "fin_heb"
filetype = "gold" # parallel_sample | gold
# filetype = "parallel_sample"
targ_lang = lang_pair.split("_")[1]
target_folder = f"alternative_data/{lang_pair}/source_excel_{filetype}"

In [ ]:
with open(f"alternative_data/{lang_pair}/{filetype}_augmented.json", "r") as f:
    sentences = json.load(f)

real_sentences = []
temp_ids = set()
for sent in sentences:
    if sent["sentence_id"] not in temp_ids:
        temp_ids.add(sent["sentence_id"])
        real_sentences.append({"sentence":sent["masked_sentence"].replace("[MASK]", sent["word"]),
                              "sentence_id": sent["sentence_id"],
                              "word_id": -1,
                              "probability": 0})

for sent in sentences:
    ct = 0
    for rep in sent["replacement"]:
        if rep["word"].strip().lower()!=sent["word"].lower():
            ct+=1
            real_sentences.append({"sentence":sent["masked_sentence"].replace("[MASK]", rep["word"].strip()),
                                  "sentence_id": sent["sentence_id"],
                                  "word_id": sent["word_id"],
                                  "probability": rep["prob"]})
selected_sentences = sorted(real_sentences, key=lambda d: (d['sentence_id'], d["word_id"])) 

In [ ]:
df_en = pd.DataFrame([])
i = 0
while True:
    try:
        df_temp = pd.read_excel(f"{target_folder}/{i}.xlsx", header=None)
    except Exception as e:
        print(e)
        print(f"Breaking at {i}")
        break
    df_en = pd.concat([df_en, df_temp])
    i+=1

In [ ]:
len(df_en) 

In [ ]:
df = pd.DataFrame([])
i=0
while True:
    try:
        df_temp = pd.read_excel(f"{target_folder}/{i}_{targ_lang}.xlsx", header=None)
    except Exception as e:
        print(e)
        print(f"Breaking at {i}")
        break
    df = pd.concat([df, df_temp])
    i+=1

In [ ]:
len(df)

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
import re, string
target_sentences = [
re.sub(r"\s+", r" ", re.sub(r'([{}])'.format("”“!\"#$%&()*+,-./:;<=>?@[\\]^_`{|}~"), r' \1 ',str(x).strip()))
    for x in df[0].tolist()]

In [ ]:
len(selected_sentences), len(target_sentences), len(df), len(df_en)

In [ ]:
target_sentences[:3]

In [ ]:
selected_sentences[:3]

In [ ]:
selected_sentences[0]

In [ ]:
for el, ts in zip(selected_sentences, target_sentences):
    el["translated_sentences"] = ts

In [ ]:
selected_sentences[2]

In [ ]:
with open(f"alternative_data/{lang_pair}/{filetype}_translated_pairs.json", "w") as f:
    json.dump(selected_sentences, f)